In [2]:
import os
import mykeys


In [3]:
mykeys.setOsEnv()

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.
mykey => set HUGGINGFACEHUB_API_TOKEN is done
mykey => set OPENAI_API_KEY is done
mykey => set LANGCHAIN_API_KEY is done
mykey => set LANGCHAIN_HUB_API_KEY is done
mykey => set GOOGLE_API_KEY is done
mykey => set GOOGLE_CSE_ID is done
mykey => set UPSTAGE_API_KEY is done
mykey => set COHERE_API_KEY is done
mykey => set JINA_API_KEY is done
mykey => set ANTHROPIC_API_KEY is done
mykey => set DEEPL_API_KEY is done
mykey => set TAVILY_API_KEY is done
mykey => set TOGETHER_API_KEY is done


In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import openai
import json
import os
import time
import concurrent.futures

# --- 1. 설정: API 키 및 모델 설정 ---

# (중요!)
# 스크립트 실행 전, 터미널에서 API 키를 설정해야 합니다.
# export OPENAI_API_KEY="sk-..."
try:
  client = openai.OpenAI()
  if not client.api_key:
    raise openai.OpenAIError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
except Exception as e:
  print(f"API 키 로드 오류: {e}")
  exit()

GPT_API_NAME = "gpt-4.1"
# (중요!) 동시 작업 스레드 수. 
# 너무 높으면 API 속도 제한이나 IP 차단 위험이 있습니다. 5~10이 적절합니다.
MAX_WORKERS = 1

# (중요!) 스크래핑 차단을 피하기 위한 필수 User-Agent
REQUEST_HEADERS = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

# --- 2. 헬퍼 함수 정의 ---

def scrape_url(url):
  """
  상세 URL을 스크래핑하고 정제된 텍스트를 반환합니다.
  """
  try:
    response = requests.get(url, headers=REQUEST_HEADERS, timeout=10)
    response.raise_for_status() # 오류가 있으면 예외 발생

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # script, style, nav, footer 등 불필요한 태그 제거
    for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
      tag.decompose()
      
    # 모든 텍스트를 공백으로 연결하여 추출
    text = soup.get_text(separator=' ', strip=True)
    
    # LLM의 토큰 제한을 피하기 위해 텍스트 길이 제한 (약 4000자)
    max_chars = 4000
    if len(text) > max_chars:
      text = text[:max_chars] + "... (내용이 너무 길어 생략됨)"
      
    return text
  
  except requests.RequestException as e:
    print(f"  > [오류] URL 스크래핑 실패 (URL: {url}): {e}")
    return "스크래핑 실패: URL에 접근할 수 없습니다."

In [14]:
def format_prompt(store_row, menu_group, scraped_text):
  """
  LLM에 전달할 프롬프트를 생성합니다.
  (수정됨: 3개 항목을 포함한 JSON을 요청)
  """
  
  # 1. store_df 정보
  store_json = store_row.to_json(force_ascii=False, indent=2)
  
  # 2. menu_df 정보 (필요한 컬럼 확장)
  menu_columns = ['메뉴', '가격원문', '가격범위', '테마', '온도', '카테고리', '주재료', '맵기(O/X)']
  valid_menu_cols = [col for col in menu_columns if col in menu_group.columns]
  menu_json = menu_group[valid_menu_cols].to_json(orient='records', force_ascii=False, indent=2)

  # 3. 시스템 프롬프트
  system_prompt = """
  당신은 레스토랑 정보를 분석하고 요약하는 최고의 '데이터 분석가'입니다.
  당신의 임무는 3가지 소스를 분석하여, (1)상세 요약문, (2)RAG 검색용 텍스트, (3)구조화된 메타데이터를 포함하는 단일 JSON 객체를 반환하는 것입니다.
  """
  
  # 4. 유저 프롬프트 (JSON 형식으로 3가지 결과물 요청)
  # (★★★ 수정됨: metadata_json에 3개 항목 추가 및 f-string 괄호 이스케이프 ★★★)
  user_prompt = f"""
  다음은 하나의 레스토랑에 대한 3가지 데이터 소스입니다.

  [소스 1: 가게 기본 정보 (store_df)]
  {store_json}

  [소스 2: 가게 메뉴 정보 (menu_df)]
  {menu_json}

  [소스 3: 가게 상세 URL 스크랩 텍스트]
  {scraped_text}

  ---
  [작업 요청]
  위 3가지 소스를 면밀히 분석하여, *반드시* 다음 3개의 키를 가진 단일 JSON 객체를 반환해주세요.
  (AI는 response_format={{"type": "json_object"}} 형식을 준수해야 합니다.)

  {{
    "llm_summary": "(여기에 Charlie님이 요청한 기존의 상세한 '사람이 읽는' 요약문을 생성합니다. [소스 1, 2, 3]의 모든 필수 정보를 포함합니다.)",
    
    "rag_embedding_text": "(여기에 RAG 검색용 '분위기/정체성' 텍스트를 생성합니다. [소스 1]의 '소개', '라벨', [소스 2]의 '테마', [소스 3]의 '태깅/분위기'를 조합합니다. 예: '샥스핀 전문 고급 중식당으로, 전좌석 프라이빗룸이라 조용하고 사적인 모임, 상견례, 비즈니스 다이닝에 적합합니다.'처럼 서술합니다. 가격, 맵기(O/X), 주소 등 필터링 '사실' 데이터는 *제외*합니다.)",
    
    "metadata_json": {{
      "budget_range": "([소스 2]의 '가격범위' 또는 '가격원문'을 분석하여 '저', '중', '고'로 정확히 판단)",
      "spicy_available": "([소스 2]의 '맵기(O/X)' 컬럼에 'O'가 하나라도 있으면 'O', 하나도 없으면 'X')",
      "vegetarian_options": "([소스 2]의 '주재료'나 '메뉴'에 '야채', '두부', '비건', '채식' 키워드가 있으면 'O', 없으면 'X')",
      "main_ingredients_list": "([소스 2]의 '주재료' 컬럼에서 고유한 값들을 리스트로 추출. 예: ['돼지고기', '김치', '원두'])",
      "low_level_category": "([소스 1]의 '카테고리' 값. 예: '돼지고기구이, 소고기구이')",
      "suitable_for": "([소스 1]의 '라벨'과 [소스 3]의 '분위기'를 조합하여 키워드 리스트 생성. 예: ['데이트', '조용한', '가족모임', '가성비'])",
      "food_type_list": "([소스 2]의 '카테고리' 컬럼(예: '국물류', '고기류')에서 고유한 값들을 리스트로 추출)",
      "temperature_list": "([소스 2]의 '온도' 컬럼(예: 'Hot', 'Ice', 'Ice/Hot')에서 고유한 값들을 리스트로 추출)",
      "high_level_category": "([소스 1]의 '카테고리'와 [소스 2]의 '메뉴'/'카테고리'를 종합하여 *반드시* '한식', '중식', '일식', '양식', '동남아', '디저트류', '기타' 중 하나로 *추론*하여 판단)"
    }}
  }}
  """
  
  return system_prompt, user_prompt

In [15]:
def get_llm_summary(system_prompt, user_prompt):
  """
  GPT-4.1 API를 호출하여 *JSON 객체(dict)*를 받아옵니다.
  """
  try:
    response = client.chat.completions.create(
      model=GPT_API_NAME,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ],
      # (★★★ 수정됨 ★★★)
      # AI가 JSON 객체를 반환하도록 강제
      response_format={"type": "json_object"}, 
      temperature=0.5
    )
    # (★★★ 수정됨 ★★★)
    # 텍스트가 아닌 JSON 객체를 파싱하여 dict로 반환
    result_dict = json.loads(response.choices[0].message.content)
    return result_dict
  
  except Exception as e:
    print(f"  > [오류] LLM API 호출/JSON 파싱 실패: {e}")
    # (★★★ 수정됨 ★★★)
    # 오류 시에도 동일한 3개 키 구조로 반환 (데이터프레임 오류 방지)
    return {
      "llm_summary": f"요약 생성 실패: {e}",
      "rag_embedding_text": "",
      "metadata_json": {}
    }

def process_restaurant(store_row, menu_group):
  """
  레스토랑 1개에 대한 전체 파이프라인 (스크래핑 + LLM)
  """
  
  store_id = store_row['id']
  store_name = store_row['가게']
  url = store_row['상세URL']
  
  print(f"--- 작업 시작 (ID: {store_id}, 가게: {store_name}) ---")
  
  # 1. 스크래핑
  print(f"  > [1/3] 스크래핑 시작: {url}")
  scraped_text = scrape_url(url)
  
  # 2. 프롬프트 생성
  print(f"  > [2/3] 프롬프트 생성 중...")
  system_prompt, user_prompt = format_prompt(store_row, menu_group, scraped_text)
  
  # 3. LLM 요약 (★★★ 수정됨 ★★★)
  print(f"  > [3/3] LLM ({GPT_API_NAME}) JSON 요청 중...")
  # 이제 summary_result는 3개의 키를 가진 dict임
  summary_result_dict = get_llm_summary(system_prompt, user_prompt)
  
  print(f"--- 작업 완료 (ID: {store_id}) ---")
  
  # (★★★ 수정됨 ★★★)
  # 메인 스레드로 5개 값을 반환
  return (
    store_id, 
    store_name, 
    summary_result_dict.get('llm_summary'),
    summary_result_dict.get('rag_embedding_text'),
    summary_result_dict.get('metadata_json') # 딕셔너리(JSON) 자체를 반환
  )

In [16]:
# --- 3. 메인 실행 함수 ---
def main():
  # (★★★ 1. count 변수 설정 ★★★)
  # (경고: 7713개 전체를 실행하면 17시간+ 및 막대한 API 비용이 발생합니다)
  COUNT_TO_PROCESS = 100 # (테스트할 개수. 0 또는 7713으로 설정 시 전체 실행)

  # (★★★ 2. 실제 데이터 로드 ★★★)
  # (Charlie님이 이 파일 경로를 실제 경로로 수정해야 합니다)
  try:
    print("데이터 로드 중: store_df, menu_df ...")
    # (파일 경로는 실제 환경에 맞게 수정하세요)
    store_df = pd.read_csv('./20251016_서울시_음식점_목록_GPS.csv')
    menu_df = pd.read_csv('./20251017_TOTAL_MENU.csv')
    print(f"로드 완료: store_df {store_df.shape}, menu_df {menu_df.shape}")
  except FileNotFoundError as e:
    print(f"[오류] 파일 로드 실패: {e}")
    print("스크립트와 동일한 폴더에 'store.csv'와 'menu.csv' 파일이 있는지 확인하세요.")
    return # 파일 없으면 중단

  # (★★★ 3. count 변수값에 맞게 선택 ★★★)
  if COUNT_TO_PROCESS > 0 and COUNT_TO_PROCESS < len(store_df):
    print(f"총 {len(store_df)}개 중 {COUNT_TO_PROCESS}개의 *랜덤 샘플*을 선택합니다.")
    # (★★★ 수정됨: .head() 대신 .sample() 사용 ★★★)
    # random_state=42는 테스트 시마다 동일한 샘플을 뽑기 위함 (재현성)
    store_df_sample = store_df.sample(n=COUNT_TO_PROCESS, random_state=42).copy()
  else:
    print(f"총 {len(store_df)}개 전체를 처리합니다.")
    store_df_sample = store_df.copy()
    
  # (이하 로직은 이전과 동일)

  # menu_df를 식당ID별로 미리 그룹화 (효율성)
  menu_groups = menu_df.groupby('식당ID')
  
  # 결과를 저장할 리스트
  summaries_results = []
  
  print(f"총 {len(store_df_sample)}개 레스토랑 요약 작업을 시작합니다. (최대 작업자 수: {MAX_WORKERS})")
  
  # ThreadPoolExecutor를 사용한 병렬 처리
  with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # (id, row)로 반복하며 작업(future)을 제출
    future_to_id = {}
    for _, store_row in store_df_sample.iterrows():
      store_id = store_row['id']
      
      try:
        # 해당 식당의 메뉴 그룹 가져오기
        menu_group = menu_groups.get_group(store_id)
      except KeyError:
        # 메뉴 정보가 없는 식당 (빈 DataFrame 생성)
        print(f"  > [경고] ID {store_id}는 메뉴 정보(menu_df)가 없습니다.")
        menu_group = pd.DataFrame(columns=menu_df.columns)
        
      # 병렬 작업 큐에 (작업 함수, 인자1, 인자2)를 넣음
      future = executor.submit(process_restaurant, store_row, menu_group)
      future_to_id[future] = store_id

    # 작업이 완료되는 순서대로 결과 처리
    for future in concurrent.futures.as_completed(future_to_id):
      store_id = future_to_id[future]
      try:
        # (★★★ 수정됨 ★★★)
        # (id, name, summary, rag_text, metadata) 튜플 반환
        result = future.result()
        summaries_results.append(result)

        # (★★★ 수정됨 ★★★)
        # 로그 확인 (result[2]가 llm_summary임)
        print("\n" + "="*60)
        print(f"✅ [로그] 요약 완료 (ID: {result[0]}, 가게: {result[1]})")
        print("="*60)
        print(result[2]) # (기존 상세 요약문 출력)
        print("="*60 + "\n")
        
      except Exception as e:
        print(f"!!! 치명적 오류 (ID: {store_id}): {e}")
        # (★★★ 수정됨 ★★★) 5개 컬럼에 맞춰 오류 튜플 반환
        summaries_results.append((store_id, "N/A", f"작업 실패: {e}", "", {}))

  print(f"\n--- 모든 작업 완료 ---")
  
  # (★★★ 수정됨 ★★★)
  # 결과(리스트)를 DataFrame으로 변환 (컬럼 5개)
  summary_df = pd.DataFrame(
    summaries_results, 
    columns=['id', '가게_임시', 'LLM요약', 'RAG텍스트', '메타데이터']
  )
  
  # (★★★ 수정됨 ★★★)
  # 원본 store_df_sample에 3개 컬럼을 병합
  final_sample_df = pd.merge(
    store_df_sample, 
    summary_df[['id', 'LLM요약', 'RAG텍스트', '메타데이터']], 
    on='id', 
    how='left'
  )

  print("\n--- 최종 결과 (샘플) ---")
  #print(final_sample_df[['id', '가게', 'LLM요약', 'RAG텍스트', '메타데이터']])
  
  # 파일로 저장 (파일명에 count 반영)
  output_filename = f"restaurant_summaries_output_{len(final_sample_df)}.csv"
  final_sample_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
  print(f"결과가 '{output_filename}' 파일로 저장되었습니다.")


if __name__ == "__main__":
  main()

데이터 로드 중: store_df, menu_df ...
로드 완료: store_df (7713, 14), menu_df (54625, 11)
총 7713개 중 100개의 *랜덤 샘플*을 선택합니다.
총 100개 레스토랑 요약 작업을 시작합니다. (최대 작업자 수: 1)
--- 작업 시작 (ID: 43346, 가게: 녹) ---
  > [1/3] 스크래핑 시작: https://www.bluer.co.kr/restaurants/43346
  > [2/3] 프롬프트 생성 중...
  > [3/3] LLM (gpt-4.1) JSON 요청 중...
--- 작업 완료 (ID: 43346) ---
--- 작업 시작 (ID: 36733, 가게: 금성관나주곰탕) ---
  > [1/3] 스크래핑 시작: https://www.bluer.co.kr/restaurants/36733

✅ [로그] 요약 완료 (ID: 43346, 가게: 녹)
녹은 서울 중구 신당동에 위치한 프리미엄 고깃집으로, 2024년 3월 7일에 개업했습니다. 이곳은 다시마로 숙성한 1등급 암퇘지만을 사용하여 돼지고기의 다양한 부위를 제공하며, 소고기 메뉴도 함께 준비되어 있습니다. 모든 고기는 연탄불에서 직원이 직접 구워 불향을 더해주며, 고기와 잘 어울리는 비법 양념장 냉쫄면이 인기입니다. 대표 메뉴로는 목살, 삼겹살, 가브리살, 녹돼지한판, 꽃등심, 육회 등이 있으며, 비빔쫄면, 냉쫄면, 차돌된장술밥 등 식사 메뉴도 마련되어 있습니다. 가격대는 8,000원부터 59,000원까지로 중간~높은 수준입니다. 매장은 연중무휴로 운영되며, 영업시간은 매일 11:30~23:00(마지막 주문 22:00)입니다. 전 좌석은 일반 홀이며, 프라이빗룸이나 별도의 라벨링, 레드리본 등은 없습니다. 국내산 프리미엄 돈육을 사용해 최고의 퀄리티를 추구하는 곳으로, 아직 공식 리뷰는 없으나, 고기와 면 요리 모두에서 긍정적인 평가가 기대됩니다.

  > [2/3] 프롬프트 생성 중...
  > [3/3] LLM (gpt-4.1) 